## Introduction

This notebook shows a clean ETL workflow for real geochemical data. Raw assay and mass‑spec files are messy, so the goal is to turn them into consistent, analysis‑ready tables using Python and pandas. The result is a reproducible pipeline suitable for geoscience analytics, PCA, clustering, and machine‑learning workflows.

## ETL : Extract > Transform > Load:

**Extract:** read raw files (CSV/Excel/SQL) into pandas

**Transform:** clean columns, types, missing values, joins, filters

**Load:** save clean tables (CSV/Parquet/SQL) for analysis or ML

**`EXTRACT`** The following function provides a unified, flexible way to load raw geochemical and mass‑spec files, automatically selecting the correct parser for each format and preparing the data for the transformation steps that follow.

In [1]:

from pathlib import Path
import pandas as pd

def extract_file(path, *, delimiter=None, encoding=None, **read_kwargs):
    """
    Load a tabular file into a DataFrame.
    
    Parameters
    ----------
    path : str | Path
        File path.
    delimiter : str | None
        Optional delimiter for text files (.txt, .tsv). If None, a sensible default is used.
    encoding : str | None
        Optional file encoding (e.g., 'utf-8', 'latin1', 'utf-16').
    **read_kwargs :
        Extra keyword args passed to pandas readers (e.g., dtype=..., na_values=..., engine=...).
    """
    p = Path(str(path).strip())  # strip whitespace
    ext = p.suffix.lower()

    try:
        if ext == ".csv":
            return pd.read_csv(p, encoding=encoding, **read_kwargs)

        elif ext in (".xlsx", ".xlsm", ".xltx", ".xltm"):
            # Modern Excel formats → openpyxl is typically required
            return pd.read_excel(p, engine=read_kwargs.pop("engine", "openpyxl"),
                                 dtype=read_kwargs.pop("dtype", None),
                                 **read_kwargs)

        elif ext == ".xls":
            # Legacy Excel → xlrd may be required (and may not be installed)
            return pd.read_excel(p, engine=read_kwargs.pop("engine", "xlrd"),
                                 dtype=read_kwargs.pop("dtype", None),
                                 **read_kwargs)

        elif ext == ".tsv":
            return pd.read_csv(p, sep="\t" if delimiter is None else delimiter,
                               encoding=encoding, **read_kwargs)

        elif ext == ".txt":
            # Default to tab if not specified; allow override via `delimiter`
            sep = "\t" if delimiter is None else delimiter
            return pd.read_csv(p, sep=sep, encoding=encoding, **read_kwargs)

        else:
            raise ValueError(f"Unsupported file type: {ext} for path {p}")

    except Exception as e:
        # Add context; re-raise for upstream handling/logging
        raise RuntimeError(f"Failed to parse file '{p}' (ext='{ext}'). "
                           f"Consider specifying delimiter/encoding/engine. "
                           f"Original error: {type(e).__name__}: {e}") from e


### File Extraction Utility
This function provides a flexible and robust way to load tabular data from a variety of common geoscience and laboratory file formats. It supports CSV, multiple Excel formats, TSV, and general text files, while allowing optional control over delimiters, encodings, and additional pandas reader arguments. The goal is to offer a single, consistent entry point for reading raw assay, mass‑spectrometry, or metadata files regardless of their source or formatting differences.

The function automatically detects the file extension, applies an appropriate pandas reader, and includes clear error messaging when parsing fails. This makes it suitable for ETL pipelines where input files may vary in structure or require custom parsing options.

The following snippet is an example of how to use the function; it is for demonstration only and does not need to be executed.

```python
from pathlib import Path

# Example usage (illustrative only)
file_path = Path("data/raw/assays_raw.csv")
df = extract_file(file_path)
df.head()
```


**`TRANSFORM`** Provides reusable data‑cleaning utilities for ETL workflows. Includes column standardization, dtype fixes, missing‑value handling,
and simple filtering logic.

In [2]:
import pandas as pd


def transform_assays(df: pd.DataFrame) -> pd.DataFrame:
  
    df = df.copy()

    # ---------------------------------------------------------
    # 1. Standardize column names
    # ---------------------------------------------------------
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )

    # ---------------------------------------------------------
    # 2. Remove fully empty rows/columns
    # ---------------------------------------------------------
    df = df.dropna(how="all")
    df = df.dropna(axis=1, how="all")

    # ---------------------------------------------------------
    # 3. Convert numeric columns safely
    # ---------------------------------------------------------
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = pd.to_numeric(df[col], errors="ignore")

    # ---------------------------------------------------------
    # 4. Handle missing values (simple example)
    # ---------------------------------------------------------
    df = df.fillna(value={"sample_id": "UNKNOWN"})

    # ---------------------------------------------------------
    # 5. Optional filtering logic (example only)
    # ---------------------------------------------------------
    if "au_ppm" in df.columns:
        df = df[df["au_ppm"] >= 0]  # remove negative gold grades

    return df


The transform module demonstrates:
- Column name normalization
- Removing empty rows/columns
- Safe numeric conversion
- Missing‑value handling
- Example domain logic (e.g., removing negative Au grades)
- A single, clean transform_assays() function


**Transform Module (ICP‑MS Friendly- multi-element datasets)**  

In [3]:
"""
TRANSFORM MODULE
----------------
Provides reusable, well‑documented cleaning utilities for multi‑element
geochemistry (ICP‑MS) datasets. Each step is intentionally simple,
transparent, and easy to adapt for real workflows.

Example usage (illustrative only — replace with your own DataFrame):

    from transform import transform_geochem
    df_clean = transform_geochem(df_raw)
    df_clean.head()
"""

import pandas as pd
import numpy as np


def transform_geochem(df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply standard cleaning steps to a multi‑element geochemistry dataset.

    Parameters
    ----------
    df : pd.DataFrame
        Raw input DataFrame.

    Returns
    -------
    pd.DataFrame
        Cleaned and standardized DataFrame.
    """

    # Work on a copy to avoid modifying the original
    df = df.copy()

    # ---------------------------------------------------------
    # 1. Standardize column names
    # ---------------------------------------------------------
    # Geochem datasets often come with inconsistent naming:
    # "Sample ID", "sample-id", "SAMPLEID", etc.
    # This normalizes everything to snake_case.
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )

    # ---------------------------------------------------------
    # 2. Remove fully empty rows/columns
    # ---------------------------------------------------------
    # ICP-MS exports sometimes include blank rows at the bottom OR empty columns used as separators.
    df = df.dropna(how="all")
    df = df.dropna(axis=1, how="all")

    # ---------------------------------------------------------
    # 3. Convert numeric columns safely
    # ---------------------------------------------------------
    # Many ICP-MS labs export numbers as strings, sometimes with
    # "<0.01" or "BDL" (below detection limit). We convert what we can.
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = pd.to_numeric(df[col], errors="ignore")

    # ---------------------------------------------------------
    # 4. Handle detection limits (e.g., "<0.01")
    # ---------------------------------------------------------
    # Replace strings like "<0.01" with half the detection limit (0.005).
    # This is a common geochem practice for statistical work.
    for col in df.columns:
        if df[col].dtype == "object":
            mask = df[col].astype(str).str.startswith("<")
            if mask.any():
                # Extract numeric part after "<"
                dl = (
                    df.loc[mask, col]
                    .str.replace("<", "", regex=False)
                    .astype(float)
                )
                df.loc[mask, col] = dl / 2  # DL/2 substitution

    # After replacing "<DL" values, try numeric conversion again
    df = df.apply(pd.to_numeric, errors="ignore")

    # ---------------------------------------------------------
    # 5. Replace negative concentrations
    # ---------------------------------------------------------
    # Negative values appear when ICP-MS subtracts blank drift.
    # They are not physically meaningful, so we set them to zero.
    numeric_cols = df.select_dtypes(include="number").columns
    for col in numeric_cols:
        df[col] = df[col].clip(lower=0)

    # ---------------------------------------------------------
    # 6. Handle missing sample IDs
    # ---------------------------------------------------------
    if "sample_id" in df.columns:
        df["sample_id"] = df["sample_id"].fillna("UNKNOWN")

    # ---------------------------------------------------------
    # 7. Outlier removal using Median Absolute Deviation (MAD)
    # ---------------------------------------------------------
    # This is optional but useful for ICP-MS datasets with spikes.
    def remove_outliers_mad(series: pd.Series, threshold: float = 5.0):
        if not np.issubdtype(series.dtype, np.number):
            return series
        median = series.median()
        mad = np.median(np.abs(series - median))
        if mad == 0:
            return series
        z = 0.6745 * (series - median) / mad
        return series.where(np.abs(z) < threshold)

    df[numeric_cols] = df[numeric_cols].apply(remove_outliers_mad)

    # ---------------------------------------------------------
    # 8. Optional compositional transforms (CLR)
    # ---------------------------------------------------------
    # CLR is useful for multivariate geochem analysis.
    # We apply it only to numeric columns and only if all values > 0.
    def clr_transform(df_numeric: pd.DataFrame) -> pd.DataFrame:
        # CLR requires strictly positive values
        if (df_numeric <= 0).any().any():
            return df_numeric
        geometric_mean = df_numeric.apply(lambda row: np.exp(np.mean(np.log(row))), axis=1)
        return df_numeric.div(geometric_mean, axis=0).apply(np.log)

    # Example: store CLR-transformed data in a separate block
    # (not replacing the original)
    df_clr = clr_transform(df[numeric_cols])
    df_clr.columns = [f"{c}_clr" for c in df_clr.columns]

    # Attach CLR columns to the dataset
    df = pd.concat([df, df_clr], axis=1)

    return df


##### Transform Stage
The Transform step applies a series of cleaning and preprocessing operations to prepare raw ICP‑MS geochemistry data for analysis. The goal is to make the dataset consistent, numeric, and statistically usable while preserving geochemical meaning.

This stage includes:

- Column name standardization
- Removal of empty rows/columns
- Safe numeric conversion
- Detection‑limit handling (<0.01 → 0.005)
- Negative value correction
- Missing value handling
- Outlier removal using MAD
- Optional CLR compositional transformation

These steps are intentionally simple, transparent, and easy to adapt for real geochemistry workflows.

1. Standardize Column Names
Geochemistry datasets often contain inconsistent naming conventions such as "Sample ID", "sample-id", or "SAMPLEID".
We convert everything to snake_case:

- lowercase
- spaces  
- hyphens 

This makes downstream processing predictable.

2. Remove Empty Rows and Columns
ICP‑MS exports sometimes include blank rows at the bottom or empty separator columns. These are removed to keep the dataset clean.

3. Convert Numeric Columns Safely
Many labs export numbers as strings, sometimes mixed with text like "BDL" or "<0.01". We attempt numeric conversion without breaking non‑numeric columns.

4. Handle Detection Limits (<DL)
Values like "<0.01" are common in ICP‑MS data. We replace them with half the detection limit:

"<0.01" → 0.005

This is a widely used geochemical practice for statistical analysis.

5. Replace Negative Concentrations
Negative values occur when blank drift correction overshoots. They are not physically meaningful, so we set them to zero.

6. Handle Missing Sample IDs
If a sample_id column exists, missing values are replaced with "UNKNOWN".

7. Remove Outliers Using MAD
The Median Absolute Deviation (MAD) method removes extreme spikes without assuming a normal distribution. This is safer for geochemical data than z‑scores.

8. Optional CLR Transformation
The Centred Log‑Ratio (CLR) transform is useful for multivariate geochemical analysis. We apply CLR only when:
- all values are positive
- the column is numeric

CLR‑transformed columns are added with a _clr suffix.


**`Load Stage`**
The Load step finalizes the ETL workflow by saving the cleaned dataset into a consistent, analysis‑ready format.
This stage ensures that:

- The processed data is reproducible
- Downstream notebooks can load it easily
- The file structure remains clean and predictable


In [4]:
"""
LOAD MODULE
-----------
Handles saving cleaned datasets to disk in a consistent, safe, and
reproducible way.

Example usage (illustrative only — replace with your own DataFrame):

    from load import load_data
    load_data(df_clean, "data/processed/assays_clean.csv")
"""

from pathlib import Path
import pandas as pd


def load_data(df: pd.DataFrame, output_path: str | Path) -> None:
    """
    Save a cleaned DataFrame to disk in CSV format.

    Parameters
    ----------
    df : pd.DataFrame
        Cleaned dataset to save.
    output_path : str or Path
        Destination file path (e.g., 'data/processed/output.csv').

    Raises
    ------
    RuntimeError
        If saving fails for any reason.
    """

    output_path = Path(output_path)

    # ---------------------------------------------------------
    # 1. Ensure the output directory exists
    # ---------------------------------------------------------
    if not output_path.parent.exists():
        output_path.parent.mkdir(parents=True, exist_ok=True)

    # ---------------------------------------------------------
    # 2. Save the DataFrame safely
    # ---------------------------------------------------------
    try:
        df.to_csv(output_path, index=False, encoding="utf-8")
    except Exception as e:
        raise RuntimeError(
            f"Failed to save file to '{output_path}'. "
            f"Original error: {e}"
        )


The load_data() function completes the ETL pipeline by:
- Creating output directories automatically
- Saving cleaned datasets in a consistent format
- Ensuring reproducibility for downstream analysis
- Providing clear, user‑friendly error messages


## Summary
This project provides a small, modular ETL framework written in Python to demonstrate how geochemistry data can be processed cleanly and reproducibly. The code is organized into three simple modules: one for extracting raw files, one for transforming and cleaning the data, and one for saving the final output. The transform step includes geochemistry‑aware logic such as handling detection limits, fixing numeric values, removing outliers, and adding optional CLR features. Each module is intentionally lightweight, readable, and easy to adapt to other datasets. The goal is to give users a clear example of how to build a professional ETL workflow in Python without unnecessary complexity.